In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
train_url = '/kaggle/input/bateclacor/train_corr.csv'
test_url = '/kaggle/input/bateclacor/test_corr.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [29]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [30]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [31]:
df_test

,Unnamed: 0,Video ID,Channel name,Time of Publishing,Title,Genre,Comment,Error,Category,Correct form
0,0,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,গান শুনলে মন ভরে,0,NaN,এই সব গান শুনলে মন ভরে যায়
1,1,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,মনটা ভরে গেলো ভাই।অনেক সুন্দর,0,NaN,মনটা ভরে গেলো ভাই।অনেক সুন্দর হয়েছে
2,2,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,বাংলা গানটা শুনছি..,0,NaN,বাংলা সেরা গানটা আবার শুনছি..
3,3,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,সুমন রায় কন্ঠে মে গান ভালো লাগে।তবে কালিয়ার ...,0,NaN,সুমন রায় এর কন্ঠে মে কোন গান ই আমার অনেক ভালো...
4,4,nkG0RZmUlmU,Banglar Gayen,2022-06-17T11:00:23Z,Jodi Bondu Jabar Chaw | যদি বন্ধু যাবার চাও | ...,Entertainment,রতীন্দ্রনাত দাদুর কন্ঠে সবচেয়ে লাগে গানটি শুনতে।,0,NaN,রতীন্দ্রনাত দাদুর কন্ঠে সবচেয়ে বেশি ভাল লাগে এ...
...,...,...,...,...,...,...,...,...,...,...
5017,5017,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,নাসির হোসেন বর্তমানে বাংলাদেশের একটা ভালো অলরা...,1,Spelling,নাসির হোসেন বর্তমানে বাংলাদেশের একটা ভালো অলরা...
5018,5018,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,তামিম ইকবাল ওনার না এশিয়া কাপ খেলার কথা �,1,Spelling,তামিম ইকবাল কোথায় উনার না এশিয়া কাপ থেকে খেল...
5019,5019,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,লিটন এন্ড তাওহীদ রিদয় কই কেমন টিম ����,1,Code Switching,লিটন এন্ড তাওহীদ রিদয় কই কেমন দল হলো এটা ����
5020,5020,piO85BnIJBc,Channel 24,2022-08-13T12:43:20Z,মাহমুদউল্লাহ-সাব্বিরকে নিয়ে এশিয়া কাপের দল ঘ...,Sports,"সাব্বির একটা ম্যাচ ভালো খেলে না,,,কেন নে ওরে,,��",1,Spelling,"সাব্বির একটা ম্যাচ ও ভালো খেলে না,,,কেন নেয় ওক..."


In [35]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Error)
Test_Y = Encoder.fit_transform(df_test.Error)

In [62]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
#Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect = TfidfVectorizer(max_features=10000, norm='l2')

Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [39]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')


SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5295    0.3435    0.4166      1910
           1     0.6685    0.8127    0.7336      3112

    accuracy                         0.6342      5022
   macro avg     0.5990    0.5781    0.5751      5022
weighted avg     0.6156    0.6342    0.6130      5022



In [21]:
#SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4693    0.7272    0.5704      1910
           1     0.7473    0.4952    0.5957      3112

    accuracy                         0.5834      5022
   macro avg     0.6083    0.6112    0.5831      5022
weighted avg     0.6416    0.5834    0.5861      5022



In [40]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=2, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4693    0.7272    0.5704      1910
           1     0.7473    0.4952    0.5957      3112

    accuracy                         0.5834      5022
   macro avg     0.6083    0.6112    0.5831      5022
weighted avg     0.6416    0.5834    0.5861      5022



In [42]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4693    0.7272    0.5704      1910
           1     0.7473    0.4952    0.5957      3112

    accuracy                         0.5834      5022
   macro avg     0.6083    0.6112    0.5831      5022
weighted avg     0.6416    0.5834    0.5861      5022



In [43]:
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1910
           1     0.6197    1.0000    0.7652      3112

    accuracy                         0.6197      5022
   macro avg     0.3098    0.5000    0.3826      5022
weighted avg     0.3840    0.6197    0.4742      5022



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='auto')


SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5758    0.0099    0.0196      1910
           1     0.6210    0.9955    0.7648      3112

    accuracy                         0.6207      5022
   macro avg     0.5984    0.5027    0.3922      5022
weighted avg     0.6038    0.6207    0.4814      5022



In [45]:
SVM = svm.SVC(C=0.1, kernel='linear', degree=4, gamma='auto')


SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5758    0.0099    0.0196      1910
           1     0.6210    0.9955    0.7648      3112

    accuracy                         0.6207      5022
   macro avg     0.5984    0.5027    0.3922      5022
weighted avg     0.6038    0.6207    0.4814      5022



In [46]:
SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='scale')


SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5758    0.0099    0.0196      1910
           1     0.6210    0.9955    0.7648      3112

    accuracy                         0.6207      5022
   macro avg     0.5984    0.5027    0.3922      5022
weighted avg     0.6038    0.6207    0.4814      5022



In [47]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4693    0.7272    0.5704      1910
           1     0.7473    0.4952    0.5957      3112

    accuracy                         0.5834      5022
   macro avg     0.6083    0.6112    0.5831      5022
weighted avg     0.6416    0.5834    0.5861      5022



In [48]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=4, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4970    0.6026    0.5447      1910
           1     0.7195    0.6256    0.6693      3112

    accuracy                         0.6169      5022
   macro avg     0.6082    0.6141    0.6070      5022
weighted avg     0.6349    0.6169    0.6219      5022



In [ ]:
SVM = svm.SVC(C=100.0, kernel='linear', degree=3, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

In [50]:
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5177    0.4220    0.4650      1910
           1     0.6814    0.7587    0.7180      3112

    accuracy                         0.6306      5022
   macro avg     0.5995    0.5903    0.5915      5022
weighted avg     0.6191    0.6306    0.6217      5022



In [51]:
SVM = svm.SVC(C=0.1, kernel='poly', degree=3, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5089    0.1194    0.1934      1910
           1     0.6323    0.9293    0.7525      3112

    accuracy                         0.6213      5022
   macro avg     0.5706    0.5243    0.4730      5022
weighted avg     0.5854    0.6213    0.5399      5022



In [54]:
SVM = svm.SVC(C=0.1, kernel='poly', degree=4, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5040    0.0660    0.1167      1910
           1     0.6262    0.9602    0.7580      3112

    accuracy                         0.6201      5022
   macro avg     0.5651    0.5131    0.4373      5022
weighted avg     0.5797    0.6201    0.5141      5022



In [53]:
SVM = svm.SVC(C=1.0, kernel='poly', degree=2, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4985    0.5094    0.5039      1910
           1     0.6948    0.6854    0.6901      3112

    accuracy                         0.6185      5022
   macro avg     0.5966    0.5974    0.5970      5022
weighted avg     0.6201    0.6185    0.6193      5022



In [52]:
SVM = svm.SVC(C=1.0, kernel='poly', degree=4, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5155    0.3403    0.4100      1910
           1     0.6650    0.8037    0.7278      3112

    accuracy                         0.6274      5022
   macro avg     0.5902    0.5720    0.5689      5022
weighted avg     0.6081    0.6274    0.6069      5022



In [55]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4970    0.6026    0.5447      1910
           1     0.7195    0.6256    0.6693      3112

    accuracy                         0.6169      5022
   macro avg     0.6082    0.6141    0.6070      5022
weighted avg     0.6349    0.6169    0.6219      5022



In [56]:
SVM = svm.SVC(C=10.0, kernel='rbf', degree=4, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.4989    0.4738    0.4860      1910
           1     0.6867    0.7079    0.6972      3112

    accuracy                         0.6189      5022
   macro avg     0.5928    0.5909    0.5916      5022
weighted avg     0.6153    0.6189    0.6169      5022



In [57]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1910
           1     0.6197    1.0000    0.7652      3112

    accuracy                         0.6197      5022
   macro avg     0.3098    0.5000    0.3826      5022
weighted avg     0.3840    0.6197    0.4742      5022



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=2, gamma='auto', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1910
           1     0.6197    1.0000    0.7652      3112

    accuracy                         0.6197      5022
   macro avg     0.3098    0.5000    0.3826      5022
weighted avg     0.3840    0.6197    0.4742      5022



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
SVM = svm.SVC(C=0.1, kernel='rbf', degree=3, gamma='scale', class_weight='balanced')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5046    0.4304    0.4645      1910
           1     0.6793    0.7407    0.7087      3112

    accuracy                         0.6227      5022
   macro avg     0.5920    0.5855    0.5866      5022
weighted avg     0.6129    0.6227    0.6158      5022



In [63]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=4, gamma='scale')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5543    0.2832    0.3749      1910
           1     0.6616    0.8602    0.7480      3112

    accuracy                         0.6408      5022
   macro avg     0.6080    0.5717    0.5614      5022
weighted avg     0.6208    0.6408    0.6061      5022



In [64]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5543    0.2832    0.3749      1910
           1     0.6616    0.8602    0.7480      3112

    accuracy                         0.6408      5022
   macro avg     0.6080    0.5717    0.5614      5022
weighted avg     0.6208    0.6408    0.6061      5022



In [65]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='scale')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5295    0.3435    0.4166      1910
           1     0.6685    0.8127    0.7336      3112

    accuracy                         0.6342      5022
   macro avg     0.5990    0.5781    0.5751      5022
weighted avg     0.6156    0.6342    0.6130      5022



In [66]:
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='scale')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5411    0.2791    0.3682      1910
           1     0.6589    0.8548    0.7442      3112

    accuracy                         0.6358      5022
   macro avg     0.6000    0.5669    0.5562      5022
weighted avg     0.6141    0.6358    0.6012      5022



In [67]:
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1910
           1     0.6197    1.0000    0.7652      3112

    accuracy                         0.6197      5022
   macro avg     0.3098    0.5000    0.3826      5022
weighted avg     0.3840    0.6197    0.4742      5022



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
SVM = svm.SVC(C=10.0, kernel='rbf', degree=4, gamma='scale')

SVM.fit(Train_X_Tfidf,Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print(metrics.classification_report(Test_Y, predictions_SVM,digits = 4))

              precision    recall  f1-score   support

           0     0.5077    0.3984    0.4465      1910
           1     0.6739    0.7629    0.7156      3112

    accuracy                         0.6243      5022
   macro avg     0.5908    0.5806    0.5810      5022
weighted avg     0.6107    0.6243    0.6132      5022



In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
df_all = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(norm='l2')

Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

SVM = svm.SVC(class_weight='balanced')

param_grid = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4],
              'gamma': ['scale', 'auto']}

grid_search = GridSearchCV(SVM, param_grid, cv=5, scoring='accuracy')
grid_search.fit(Train_X_Tfidf, Train_Y)

# Print the best parameters
print("Best Parameters:", grid_search.best_params_)

# Use the best model for prediction
best_SVM = grid_search.best_estimator_

# Predict on the test set
predictions_test = best_SVM.predict(Test_X_Tfidf)

# Print the classification report for the test set
print(metrics.classification_report(Test_Y, predictions_test, digits=4))



KeyboardInterrupt: 